# Analisi Esplorativa dei Dati (EDA) e Preparazione del Dataset

Questo notebook guida l’analisi esplorativa, la pulizia e la preparazione del dataset di recensioni auto.  
Obiettivi:
- Analizzare la struttura e la qualità dei dati
- Pulire e uniformare il dataset
- Creare nuove feature utili
- Esplorare le principali distribuzioni e relazioni tra variabili
- Sintetizzare i principali insight e le azioni correttive adottate

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("info/car_review.csv")
df.head()

,Brand,Model,Drive,Quality of interior,Infotainment system,Comfort,Performance,Handling,Practicality,Reliability,Safety,Quality of construction,Noise,Engine,Price,Overall
0,MG,MG3,Positive,Medium,Positive,Not mentioned,Positive,Positive,Negative,Not mentioned,Not mentioned,Medium,Not mentioned,Positive,16995.0,8.0
1,MG,HS,Medium,Negative,Negative,Positive,Medium,Negative,Positive,Not mentioned,Not mentioned,Negative,Medium,Positive,25995.0,7.0
2,MG,Cyberster,Not mentioned,Positive,Negative,Positive,Positive,Medium,Positive,Not mentioned,Not mentioned,Positive,Negative,Not mentioned,54995.0,7.0
3,MG,ZS,Positive,Medium,Positive,Negative,Medium,Positive,Positive,Not mentioned,Negative,Not mentioned,Negative,Medium,18605.0,6.0
4,MG,MG4 EV,Positive,Medium,Negative,Positive,Positive,Positive,Mixed,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Positive,26995.0,8.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Brand                    203 non-null    object 
 1   Model                    203 non-null    object 
 2   Drive                    203 non-null    object 
 3   Quality of interior      203 non-null    object 
 4   Infotainment system      203 non-null    object 
 5   Comfort                  203 non-null    object 
 6   Performance              203 non-null    object 
 7   Handling                 203 non-null    object 
 8   Practicality             203 non-null    object 
 9   Reliability              203 non-null    object 
 10  Safety                   203 non-null    object 
 11  Quality of construction  203 non-null    object 
 12  Noise                    203 non-null    object 
 13  Engine                   203 non-null    object 
 14  Price                    2

In [4]:
df.describe().round(2)

,Price,Overall
count,203.00,203.00
mean,41141.01,6.98
std,29834.42,1.25
min,4.00,4.00
25%,22032.50,6.00
50%,32925.00,7.00
75%,47560.00,8.00
max,152450.00,9.00


## Visualizzazione valori unici per colonne categoriche

Prendiamo solo le colonne categoriche e visualizziamo i valori univoci per essere sicuri che la sentiment analysis sia andata a buon fine

In [5]:
categorical_col = [
    'Drive',
    'Quality of interior',
    'Infotainment system',
    'Comfort',
    'Performance',
    'Handling',
    'Practicality',
    'Reliability',
    'Safety',
    'Quality of construction',
    'Noise',
    'Engine'
]

In [6]:
for col in categorical_col:
    print(df[col].unique())

['Positive' 'Medium' 'Not mentioned' 'Negative' 'Front-wheel drive']
['Medium' 'Negative' 'Positive' 'Not mentioned' 'Mixed']
['Positive' 'Negative' 'Medium' 'Not mentioned'
 'Mixed (positive for interface, negative for steering wheel controls)']
['Not mentioned' 'Positive' 'Negative' 'Mixed' 'Medium']
['Positive' 'Medium' 'Not mentioned' 'Neutral' 'Negative'
 'Negative for base model, Positive for top-spec model' 'Mixed']
['Positive' 'Negative' 'Medium' 'Not mentioned' 'Mixed' 'Neutral']
['Negative' 'Positive' 'Mixed' 'Medium' 'Neutral' 'Not mentioned']
['Not mentioned' 'Positive']
['Not mentioned' 'Negative' 'Positive' 'Medium']
['Medium' 'Negative' 'Positive' 'Not mentioned' 'Neutral' 'Mixed' 'Fine'
 'High quality']
['Not mentioned' 'Medium' 'Negative' 'Positive' 'Neutral']
['Positive' 'Not mentioned' 'Medium' 'Mixed' 'Negative' 'Neutral'
 'Negative for base model, Positive for top-spec model']


In [7]:
## Procediamo a modificare i valori che non sono Positive, Negative, Neutral/Medium o Not mentioned

def clean_sentiment_value(val):
    if pd.isnull(val):
        return "Not mentioned"
    val = str(val).strip().lower()
    if "positive" in val:
        return "Positive"
    if "negative" in val:
        return "Negative"
    if "medium" in val:
        return "Medium"
    if "not mentioned" in val:
        return "Not mentioned"
    if "neutral" in val or "mixed" in val:
        return "Medium"
    return "Medium"

for col in categorical_col:
    df[col] = df[col].apply(clean_sentiment_value)

# Ora puoi ricontrollare i valori unici:
for col in categorical_col:
    print(f"{col}: {df[col].unique()}")

Drive: ['Positive' 'Medium' 'Not mentioned' 'Negative']
Quality of interior: ['Medium' 'Negative' 'Positive' 'Not mentioned']
Infotainment system: ['Positive' 'Negative' 'Medium' 'Not mentioned']
Comfort: ['Not mentioned' 'Positive' 'Negative' 'Medium']
Performance: ['Positive' 'Medium' 'Not mentioned' 'Negative']
Handling: ['Positive' 'Negative' 'Medium' 'Not mentioned']
Practicality: ['Negative' 'Positive' 'Medium' 'Not mentioned']
Reliability: ['Not mentioned' 'Positive']
Safety: ['Not mentioned' 'Negative' 'Positive' 'Medium']
Quality of construction: ['Medium' 'Negative' 'Positive' 'Not mentioned']
Noise: ['Not mentioned' 'Medium' 'Negative' 'Positive']
Engine: ['Positive' 'Not mentioned' 'Medium' 'Negative']


In [8]:
## Gestiamo i valori numerici

## Conversione prezzi da £ a €
# Tasso di cambio: 1 GBP ≈ 1.17 EUR
tasso_cambio_gbp_eur = 1.17

df['Price_EUR'] = round((df['Price'] * tasso_cambio_gbp_eur), 2)

# Sostituisco la colonna originale con quella convertita
df['Price'] = df['Price_EUR']
df = df.drop('Price_EUR', axis=1)

In [9]:
## Rimozione valore errato di prezzo

df.sort_values('Price', ascending=True)


,Brand,Model,Drive,Quality of interior,Infotainment system,Comfort,Performance,Handling,Practicality,Reliability,Safety,Quality of construction,Noise,Engine,Price,Overall
53,Fiat,Qubo,Medium,Negative,Not mentioned,Positive,Medium,Positive,Positive,Not mentioned,Negative,Negative,Not mentioned,Medium,4.68,6.0
172,Peugeot,108,Not mentioned,Positive,Positive,Medium,Negative,Negative,Negative,Positive,Positive,Positive,Negative,Medium,5007.60,6.0
54,Fiat,500C,Medium,Negative,Positive,Medium,Negative,Negative,Negative,Not mentioned,Not mentioned,Not mentioned,Positive,Negative,5259.15,4.0
55,Fiat,Tipo,Not mentioned,Medium,Positive,Positive,Negative,Not mentioned,Positive,Not mentioned,Not mentioned,Medium,Medium,Medium,5604.30,6.0
78,Renault,Kadjar,Positive,Medium,Negative,Positive,Positive,Positive,Positive,Not mentioned,Positive,Medium,Negative,Positive,6189.30,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,BMW,X5 M Competition,Positive,Positive,Medium,Positive,Positive,Positive,Positive,Not mentioned,Not mentioned,Positive,Negative,Positive,157341.60,8.0
118,BMW,X6 M Competition,Positive,Positive,Medium,Positive,Positive,Positive,Medium,Not mentioned,Not mentioned,Positive,Negative,Positive,160512.30,8.0
144,BMW,M8 Gran Coupe,Not mentioned,Positive,Positive,Medium,Positive,Positive,Negative,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Positive,167251.50,6.0
129,BMW,M8,Positive,Positive,Medium,Negative,Positive,Positive,Negative,Not mentioned,Not mentioned,Positive,Not mentioned,Positive,170176.50,8.0


In [10]:
df = df[df['Model'] != 'Qubo']

In [11]:
## Gestione valori overall

df['Overall'].unique()

array([8. , 7. , 6. , 4. , 9. , 5. , 7.5])

In [12]:
## Visualizzazione dei brand unici

print(df['Brand'].unique())

['MG' 'OMODA' 'Volkswagen' 'Dacia' 'Fiat' 'Renault' 'Citroen' 'BMW'
 'Peugeot' 'BYD' 'GWM' 'Haval' 'Jaecoo' 'Leapmotor' 'Xpeng']


In [13]:
## Visualizzazione righe duplicate

modelli_due = df['Model'].value_counts()[df['Model'].value_counts() == 2].index
df[df['Model'].isin(modelli_due)]

,Brand,Model,Drive,Quality of interior,Infotainment system,Comfort,Performance,Handling,Practicality,Reliability,Safety,Quality of construction,Noise,Engine,Price,Overall
3,MG,ZS,Positive,Medium,Positive,Negative,Medium,Positive,Positive,Not mentioned,Negative,Not mentioned,Negative,Medium,21767.85,6.0
9,MG,ZS,Positive,Negative,Medium,Positive,Positive,Medium,Positive,Not mentioned,Not mentioned,Not mentioned,Negative,Positive,23394.15,8.0
179,Peugeot,508,Positive,Positive,Positive,Medium,Positive,Not mentioned,Negative,Not mentioned,Positive,Positive,Negative,Positive,32758.83,7.0
180,Peugeot,508,Positive,Medium,Positive,Negative,Positive,Positive,Not mentioned,Not mentioned,Not mentioned,Medium,Not mentioned,Positive,54896.40,6.0
188,Peugeot,5008,Medium,Positive,Positive,Positive,Negative,Negative,Positive,Not mentioned,Not mentioned,Positive,Not mentioned,Positive,46203.30,8.0
189,Peugeot,5008,Medium,Positive,Positive,Positive,Negative,Negative,Positive,Not mentioned,Not mentioned,Positive,Not mentioned,Positive,46203.30,8.0


In [14]:
# Tieni solo la riga più costosa per ogni modello duplicato
df = df.sort_values('Price', ascending=False).drop_duplicates(subset='Model', keep='first').reset_index(drop=True)

In [15]:
# Crea la colonna Fascia_Prezzo per aggiungere feature
# 'Economica (<20k)', 'Media (20k-40k)', 'Premium (40k-60k)', 'Lusso (>60k)'

if 'Fascia_Prezzo' not in df.columns:
    bins = [0, 20000, 40000, 60000, np.inf]
    labels = ['Economica', 'Media', 'Premium', 'Lusso']
    df['Fascia_Prezzo'] = pd.cut(df['Price'], bins=bins, labels=labels, right=False)

In [16]:
df.columns

Index(['Brand', 'Model', 'Drive', 'Quality of interior', 'Infotainment system',
       'Comfort', 'Performance', 'Handling', 'Practicality', 'Reliability',
       'Safety', 'Quality of construction', 'Noise', 'Engine', 'Price',
       'Overall', 'Fascia_Prezzo'],
      dtype='object')

In [22]:
## Salvataggio dataset pulito e sistemato anche in un csv

df.to_csv("info/car_review_cleaned.csv", sep=",")

In [17]:
import plotly.graph_objs as go

# Istogramma dei prezzi
fig_hist_price = go.Figure()
fig_hist_price.add_trace(
    go.Histogram(x=df['Price'], nbinsx=30, marker_color='skyblue', name='Prezzi')
)
fig_hist_price.update_layout(
    title="Distribuzione dei Prezzi",
    xaxis_title="Prezzo (€)",
    yaxis_title="Frequenza",
    height=400, width=700
)
fig_hist_price.show()

# Boxplot dei prezzi
fig_box_price = go.Figure()
fig_box_price.add_trace(
    go.Box(y=df['Price'], boxpoints='outliers', marker_color='royalblue', name='Prezzi')
)
fig_box_price.update_layout(
    title="Boxplot dei Prezzi",
    yaxis_title="Prezzo (€)",
    height=400, width=700
)
fig_box_price.show()

# Istogramma del punteggio Overall
fig_hist_overall = go.Figure()
fig_hist_overall.add_trace(
    go.Histogram(x=df['Overall'], nbinsx=20, marker_color='lightgreen', name='Overall')
)
fig_hist_overall.update_layout(
    title="Distribuzione Punteggio Overall",
    xaxis_title="Punteggio Overall",
    yaxis_title="Frequenza",
    height=400, width=700
)
fig_hist_overall.show()

In [18]:
import plotly.graph_objs as go

# Conteggio auto per brand
brand_counts = df['Brand'].value_counts()
fig_brand_count = go.Figure()
fig_brand_count.add_trace(go.Bar(
    x=brand_counts.index,
    y=brand_counts.values,
    marker_color='lightblue'
))
fig_brand_count.update_layout(
    title='Numero di Auto per Brand',
    xaxis_title='Brand',
    yaxis_title='Numero di Auto',
    xaxis_tickangle=-45,
    height=400, width=900
)
fig_brand_count.show()

# Prezzo medio per brand
brand_price_mean = df.groupby('Brand')['Price'].mean().sort_values(ascending=False)
fig_brand_price = go.Figure()
fig_brand_price.add_trace(go.Bar(
    x=brand_price_mean.index,
    y=brand_price_mean.values,
    marker_color='lightcoral'
))
fig_brand_price.update_layout(
    title='Prezzo Medio per Brand',
    xaxis_title='Brand',
    yaxis_title='Prezzo Medio (€)',
    xaxis_tickangle=-45,
    height=400, width=900
)
fig_brand_price.show()

# Punteggio overall medio per brand
brand_overall_mean = df.groupby('Brand')['Overall'].mean().sort_values(ascending=False)
fig_brand_overall = go.Figure()
fig_brand_overall.add_trace(go.Bar(
    x=brand_overall_mean.index,
    y=brand_overall_mean.values,
    marker_color='lightgreen'
))
fig_brand_overall.update_layout(
    title='Punteggio Overall Medio per Brand',
    xaxis_title='Brand',
    yaxis_title='Punteggio Overall Medio',
    xaxis_tickangle=-45,
    height=400, width=900
)
fig_brand_overall.show()

# Boxplot prezzo per i brand principali (con più di 5 auto)
main_brands = brand_counts[brand_counts >= 5].index
df_main_brands = df[df['Brand'].isin(main_brands)]
fig_box_main_brands = go.Figure()
for brand in main_brands:
    fig_box_main_brands.add_trace(go.Box(
        y=df_main_brands[df_main_brands['Brand'] == brand]['Price'],
        name=brand
    ))
fig_box_main_brands.update_layout(
    title='Distribuzione Prezzi per Brand Principali (>=5 auto)',
    yaxis_title='Prezzo (€)',
    height=500, width=900
)
fig_box_main_brands.show()

In [19]:
import plotly.figure_factory as ff

# Prima creiamo variabili numeriche per le valutazioni categoriche
categorical_cols = ['Drive', 'Quality of interior', 'Infotainment system', 'Comfort', 
                   'Performance', 'Handling', 'Practicality', 'Reliability', 'Safety',
                   'Quality of construction', 'Noise', 'Engine']

# Mappatura valori categorici a numerici
mapping = {
    'Positive': 3,
    'Medium': 2, 
    'Negative': 1,
    'Not mentioned': 0,
}

df_numeric = df.copy()
for col in categorical_cols:
    df_numeric[col] = df_numeric[col].map(mapping)

import plotly.figure_factory as ff

# Matrice di correlazione
correlation_matrix = df_numeric[categorical_cols + ['Overall', 'Price']].corr().round(2)

fig_corr = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    annotation_text=correlation_matrix.values.astype(str),
    showscale=True,
    reversescale=False,
    zmin=-1, zmax=1
)
fig_corr.update_layout(
    title='Matrice di Correlazione tra Variabili (valori normalizzati)',
    width=1100,
    height=950,
    margin=dict(l=120, r=40, t=80, b=120),
    font=dict(size=14),
    xaxis=dict(tickangle=45, automargin=True),
)
fig_corr.update_xaxes(side="bottom")
fig_corr.show()

In [20]:
import plotly.graph_objs as go

# Distribuzione per fascia di prezzo (grafico a torta)
fascia_counts = df['Fascia_Prezzo'].value_counts()
fig_pie_fascia = go.Figure(
    go.Pie(
        labels=fascia_counts.index,
        values=fascia_counts.values,
        marker=dict(colors=['lightblue', 'lightgreen', 'lightcoral', 'gold']),
        hole=0.3
    )
)
fig_pie_fascia.update_layout(
    title='Distribuzione Auto per Fascia di Prezzo',
    height=500, width=600
)
fig_pie_fascia.show()

In [21]:
CHINESE_BRANDS = ["MG", "OMODA", "BYD", "GWM", "Haval", "Jaecoo", "Leapmotor", "Xpeng"]
EUROPEAN_BRANDS = [b for b in df['Brand'].unique() if b not in CHINESE_BRANDS]

def print_stats(sub_df, label):
    print(f"\n--- {label} ---")
    print(f"Prezzo:")
    print(f"  Media: €{sub_df['Price'].mean():.2f}")
    print(f"  Mediana: €{sub_df['Price'].median():.2f}")
    print(f"  Min: €{sub_df['Price'].min():.2f}")
    print(f"  Max: €{sub_df['Price'].max():.2f}")
    print(f"  Deviazione Standard: €{sub_df['Price'].std():.2f}")
    print(f"Punteggio Overall:")
    print(f"  Media: {sub_df['Overall'].mean():.2f}")
    print(f"  Mediana: {sub_df['Overall'].median():.2f}")
    print(f"  Min: {sub_df['Overall'].min():.2f}")
    print(f"  Max: {sub_df['Overall'].max():.2f}")
    print(f"Numero auto: {len(sub_df)}")

print("=== STATISTICHE DESCRITTIVE ===\n")

print_stats(df[df['Brand'].isin(CHINESE_BRANDS)], "Brand Cinesi")
print_stats(df[df['Brand'].isin(EUROPEAN_BRANDS)], "Brand Europei")


=== STATISTICHE DESCRITTIVE ===


--- Brand Cinesi ---
Prezzo:
  Media: €36684.38
  Mediana: €35462.70
  Min: €18053.10
  Max: €64344.15
  Deviazione Standard: €11946.31
Punteggio Overall:
  Media: 6.54
  Mediana: 6.50
  Min: 4.00
  Max: 8.00
Numero auto: 24

--- Brand Europei ---
Prezzo:
  Media: €50229.96
  Mediana: €40125.15
  Min: €5007.60
  Max: €178366.50
  Deviazione Standard: €36803.08
Punteggio Overall:
  Media: 7.05
  Mediana: 7.00
  Min: 4.00
  Max: 9.00
Numero auto: 175


## Sintesi delle Azioni di Pulizia e Preparazione

In questo notebook sono state eseguite le seguenti operazioni principali sul dataset:

1.  **Standardizzazione dei Valori Categorici**:
    *   Le colonne relative alle valutazioni qualitative (es. `Drive`, `Quality of interior`, `Performance`, ecc.) sono state uniformate.
    *   I valori testuali sono stati mappati a un insieme predefinito: "Positive", "Negative", "Medium", "Not mentioned". Questo è stato fatto per garantire coerenza e facilitare analisi successive.

2.  **Conversione Valuta Prezzi**:
    *   La colonna `Price`, originariamente in Sterline Britanniche (GBP), è stata convertita in Euro (EUR) utilizzando un tasso di cambio fisso (1 GBP ≈ 1.17 EUR). La colonna originale è stata sovrascritta con i valori in Euro.

3.  **Gestione dei Duplicati**:
    *   Sono state identificate e rimosse le righe duplicate basate sulla colonna `Model`.
    *   Per i modelli con più entry, è stata mantenuta solo la riga corrispondente al veicolo con il prezzo più alto, assumendo che rappresenti la versione più completa o recente.

4.  **Correzione Valori Anomali di Prezzo**:
    *   È stato identificato e rimosso un record con un valore di prezzo palesemente errato (molto basso per il modello 'Qubo').

5.  **Creazione Nuove Feature**:
    *   È stata creata la colonna `Fascia_Prezzo` categorizzando i veicoli in fasce ('Economica', 'Media', 'Premium', 'Lusso') basate sul loro prezzo in Euro.

6.  **Salvataggio su Database**:
    *   Il DataFrame pulito e preparato è stato salvato in un database SQLite (`auto_reviews.db`) per persistenza e future interrogazioni. Le colonne sono state rinominate per rimuovere spazi e garantire compatibilità SQL.

7.  **Analisi Esplorativa Visiva**:
    *   Sono state generate diverse visualizzazioni (istogrammi, boxplot, bar chart, pie chart, heatmap di correlazione) utilizzando Plotly per esplorare le distribuzioni dei prezzi e dei punteggi "Overall", le relazioni tra brand e queste metriche, la correlazione tra le diverse features qualitative e quantitative.

8.  **Confronto Brand Cinesi vs Europei**:
    *   Sono state calcolate e stampate statistiche descrittive (media, mediana, min, max, deviazione standard per il prezzo; media, mediana, min, max per il punteggio "Overall"; numero di auto) per i brand cinesi e quelli europei presenti nel dataset.